# NCBI E-utilities와 Requests를 통한 API 요청 실습

**학습목표**: Python의 `requests` 모듈을 사용해 `NCBI Entrez E-utilities`를 호출하고 소규모의 데이터 워크플로우를 직접 구축하는 방법을 배운다.

**Last updated**: 2025-11-03

**Reference**: *A General Introduction to the E-utilities* (NCBI Bookshelf, NBK25497).

### 핵심 개념

* **Entrez**: NCBI가 제공하는 생물의학 데이터베이스 통합 검색 시스템.
* **E-utilities**: 데이터를 요청하고 메타데이터를 반환하는 HTTP 기반 엔드포인트.
* **UID**: 각 데이터 레코드의 고유 식별자 (예: PubMed의 PMID).
* **많이 사용되는 흐름**: `ESearch` → UID 검색 → `ESummary` 또는 `EFetch` → 세부 정보 확인.
* **History Server**: `WebEnv`와 `query_key`를 사용해 결과 집합을 저장하고 재사용.

### 주요 엔드포인트

* `einfo.fcgi` — 데이터베이스 정보 확인.
* `esearch.fcgi` — 검색어로 UID 목록 조회.
* `esummary.fcgi` — UID 요약 정보 반환.
* `efetch.fcgi` — UID의 전체 레코드 반환.
* `elink.fcgi` — 데이터베이스 간 연관 링크 조회.
* `epost.fcgi` — UID 목록을 History 서버에 업로드.
* `egquery.fcgi` — 모든 데이터베이스에서의 전체 검색 결과 개수 조회.
* `espell.fcgi` — 검색어 맞춤법 제안.
* `ecitmatch.cgi` — 인용 정보를 바탕으로 PubMed ID(PMID) 조회.

### 사용 가이드라인

* **기본 URL**: `https://eutils.ncbi.nlm.nih.gov/entrez/eutils/`
* **애플리케이션 식별**: 요청 시 `tool`과 `email` 파라미터를 포함해야 함.
* **API 키(선택사항)**: 호출 한도 증가용. 환경 변수 `EUTILS_API_KEY`로 설정.
* **요청 속도 제한 준수**: 루프 내에서 짧은 대기 시간을 추가.

### 패키지 설치

Install optional packages for XML/JSON handling.

```bash
pip install requests xmltodict pandas
```

### curl 이용해서 API 요청하고 결과 확인하기

다음 명령어를 터미널에 복사 붙여넣기 하여 실행해봅시다
```
curl -G "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi" \
  --data-urlencode "db=pubmed" \
  --data-urlencode "term=asthma treatment"
```

### Requests 라이브러리 이용하여 API 요청하고 결과 확인하기

[requests documentation](https://requests.readthedocs.io/en/latest/api/)


In [137]:
import requests

params = {
    "db": "pubmed",
    "term": "asthma treatment",
}
r = requests.get("https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi",
                    params=params)
print("Status code:", r.status_code)

Status code: 200


In [138]:
?requests.get

Signature: requests.get(url, params=None, **kwargs)
Docstring:
Sends a GET request.

:param url: URL for the new :class:`Request` object.
:param params: (optional) Dictionary, list of tuples or bytes to send
    in the query string for the :class:`Request`.
:param \*\*kwargs: Optional arguments that ``request`` takes.
:return: :class:`Response <Response>` object
:rtype: requests.Response
File:      ~/kbiohealth_교육/Python-Advanced/.venv/lib/python3.12/site-packages/requests/api.py
Type:      function

In [139]:
dir(r)[-20:]

['cookies',
 'elapsed',
 'encoding',
 'headers',
 'history',
 'is_permanent_redirect',
 'is_redirect',
 'iter_content',
 'iter_lines',
 'json',
 'links',
 'next',
 'ok',
 'raise_for_status',
 'raw',
 'reason',
 'request',
 'status_code',
 'text',
 'url']

In [140]:
r.encoding

'UTF-8'

In [141]:
r.text

'<?xml version="1.0" encoding="UTF-8" ?>\n<!DOCTYPE eSearchResult PUBLIC "-//NLM//DTD esearch 20060628//EN" "https://eutils.ncbi.nlm.nih.gov/eutils/dtd/20060628/esearch.dtd">\n<eSearchResult><Count>130338</Count><RetMax>20</RetMax><RetStart>0</RetStart><IdList>\n<Id>41176351</Id>\n<Id>41175322</Id>\n<Id>41173504</Id>\n<Id>41173502</Id>\n<Id>41173385</Id>\n<Id>41173382</Id>\n<Id>41173303</Id>\n<Id>41172705</Id>\n<Id>41172625</Id>\n<Id>41171975</Id>\n<Id>41170702</Id>\n<Id>41170415</Id>\n<Id>41170023</Id>\n<Id>41169790</Id>\n<Id>41169391</Id>\n<Id>41168642</Id>\n<Id>41167627</Id>\n<Id>41167528</Id>\n<Id>41167401</Id>\n<Id>41166913</Id>\n</IdList><TranslationSet><Translation>     <From>asthma</From>     <To>"asthma"[MeSH Terms] OR "asthma"[All Fields] OR "asthmas"[All Fields] OR "asthma\'s"[All Fields]</To>    </Translation><Translation>     <From>treatment</From>     <To>"therapeutics"[MeSH Terms] OR "therapeutics"[All Fields] OR "treatments"[All Fields] OR "therapy"[Subheading] OR "ther

In [142]:
r.json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [133]:
dict(r.headers)

{'Date': 'Mon, 03 Nov 2025 03:38:57 GMT',
 'Server': 'Finatra',
 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload',
 'Content-Security-Policy': 'upgrade-insecure-requests',
 'Referrer-Policy': 'origin-when-cross-origin',
 'NCBI-SID': '54012009A2EB3C87_8BB8SID',
 'NCBI-PHID': '1D32B5AB8576F855000050E08DC529B7.1.1.m_1',
 'Content-Type': 'text/xml; charset=UTF-8',
 'Cache-Control': 'private',
 'content-encoding': 'gzip',
 'X-RateLimit-Limit': '3',
 'X-RateLimit-Remaining': '2',
 'Access-Control-Allow-Origin': '*',
 'Access-Control-Expose-Headers': 'X-RateLimit-Limit,X-RateLimit-Remaining',
 'Set-Cookie': 'ncbi_sid=54012009A2EB3C87_8BB8SID; domain=.nih.gov; path=/; expires=Tue, 03 Nov 2026 03:38:57 GMT',
 'X-UA-Compatible': 'IE=Edge',
 'X-XSS-Protection': '1; mode=block',
 'Keep-Alive': 'timeout=4, max=40',
 'Connection': 'Keep-Alive',
 'Transfer-Encoding': 'chunked'}

In [ ]:
print("Content-Type header:", r.headers.get("content-type"))

Content-Type header: text/xml; charset=UTF-8


In [74]:
print("\n--- Raw text body (first 300 chars) ---")
print(r.text[:300])
print("\n--- Parsed JSON (if applicable) ---")
try:
    print(r.json())
except ValueError:
    print("Response is not valid JSON")


--- Raw text body (first 300 chars) ---
{
  "message": "Requires authentication",
  "documentation_url": "https://docs.github.com/rest",
  "status": "401"
}

--- Parsed JSON (if applicable) ---
{'message': 'Requires authentication', 'documentation_url': 'https://docs.github.com/rest', 'status': '401'}


### 요청을 간편하게 만들어주는 Helper 함수 만들기

In [231]:
# Helper functions

import os, time, urllib.parse
import requests
from dotenv import load_dotenv
load_dotenv()

BASE = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/"

TOOL = os.getenv("EUTILS_TOOL", "ai-class-notebook")
EMAIL = os.getenv("EUTILS_EMAIL", "student@example.com")
API_KEY = os.getenv("EUTILS_API_KEY")  # optional

def eutils_request(endpoint, params, expect_json=True):
    """Basic request wrapper for NCBI E-utilities."""
    url = urllib.parse.urljoin(BASE, endpoint)
    params = dict(params)
    params["tool"] = TOOL
    params["email"] = EMAIL
    if API_KEY:
        params["api_key"] = API_KEY
    if expect_json:
        params.setdefault("retmode", "json")

    r = requests.get(url, params=params)
    r.raise_for_status()
    return r.json() if expect_json else r.text

In [367]:
print("TOOL =", TOOL)
print("EMAIL =", EMAIL)
print("API_KEY set?" , bool(API_KEY))

TOOL = ai-class-notebook
EMAIL = student@example.com
API_KEY set? False


## Requests 라이브러리 이용해서 E-utilities API 요청해보기

### EInfo: Database capabilities

#### PubMed

In [368]:
## EInfo: Inspect a database

# Example: PubMed metadata (fields, last update, record counts)
resp = eutils_request("einfo.fcgi", {"db": "pubmed"})
print("Database:", resp["einforesult"]["dbinfo"][0]["dbname"])
print("LastUpdate:", resp["einforesult"]["dbinfo"][0]["lastupdate"])
print("\n====== LinkInfo (First 10) ======")
for link in resp["einforesult"]["dbinfo"][0]["linklist"][:10]:
    print(f'{link["name"]}: link to {link["dbto"]} DB')
print("\n====== Field Info ======")
# Show first 10 fields
fields = resp["einforesult"]["dbinfo"][0].get("fieldlist", [])
for f in fields:
    print(f'{f["name"]}: {f["fullname"]} (termcount: {f["termcount"]})')

Database: pubmed
LastUpdate: 2025/11/02 19:14

====== LinkInfo (First 10) ======
pubmed_assembly: link to assembly DB
pubmed_bioproject: link to bioproject DB
pubmed_biosample: link to biosample DB
pubmed_biosystems: link to biosystems DB
pubmed_books_refs: link to books DB
pubmed_cdd: link to cdd DB
pubmed_clinvar: link to clinvar DB
pubmed_clinvar_calculated: link to clinvar DB
pubmed_dbvar: link to dbvar DB
pubmed_gap: link to gap DB

====== Field Info ======
ALL: All Fields (termcount: )
UID: UID (termcount: )
FILT: Filter (termcount: )
TITL: Title (termcount: )
MESH: MeSH Terms (termcount: )
MAJR: MeSH Major Topic (termcount: )
JOUR: Journal (termcount: )
AFFL: Affiliation (termcount: )
ECNO: EC/RN Number (termcount: )
SUBS: Supplementary Concept (termcount: )
PDAT: Date - Publication (termcount: )
EDAT: Date - Entry (termcount: )
VOL: Volume (termcount: )
PAGE: Pagination (termcount: )
PTYP: Publication Type (termcount: )
LANG: Language (termcount: )
ISS: Issue (termcount: )
SUBH

In [369]:
pubmed_field_len = len(resp['einforesult']['dbinfo'][0]['fieldlist'])
pubmed_link_len = len(resp['einforesult']['dbinfo'][0]['linklist'])
print(f"{pubmed_field_len=}")
print(f"{pubmed_link_len=}")

pubmed_field_len=50
pubmed_link_len=57


#### Clinvar

In [370]:
## EInfo: Inspect a database

# Example: PubMed metadata (fields, last update, record counts)
resp = eutils_request("einfo.fcgi", {"db": "clinvar"})
print("Database:", resp["einforesult"]["dbinfo"][0]["dbname"])
print("LastUpdate:", resp["einforesult"]["dbinfo"][0]["lastupdate"])
print("\n====== LinkInfo (First 10)======")
for link in resp["einforesult"]["dbinfo"][0]["linklist"][:10]:
    print(f'{link["name"]}: link to {link["dbto"]} DB')
print("\n====== Field Info ======")
# Show first 10 fields
fields = resp["einforesult"]["dbinfo"][0].get("fieldlist", [])
for f in fields:
    print(f'{f["name"]}: {f["fullname"]} (termcount: {f["termcount"]})')

Database: clinvar
LastUpdate: 2025/10/27 22:26

====== LinkInfo (First 10)======
clinvar_dbvar: link to dbvar DB
clinvar_gene: link to gene DB
clinvar_gene_specific: link to gene DB
clinvar_gtr: link to gtr DB
clinvar_medgen: link to medgen DB
clinvar_omim: link to omim DB
clinvar_orgtrack: link to orgtrack DB
clinvar_pmc: link to pmc DB
clinvar_pubmed: link to pubmed DB
clinvar_pubmed_calculated: link to pubmed DB

====== Field Info ======
ALL: All Fields (termcount: 145148200)
UID: UID (termcount: 0)
FILT: Filter (termcount: 65)
TITL: Name of the ClinVar record (termcount: 5497926)
WORD: Text Word (termcount: 75279030)
ORGN: Organism (termcount: 1)
MDAT: Modification Date (termcount: 231)
CHR: Chromosome (termcount: 26)
GENE: Gene Name (termcount: 92717)
MIM: MIM (termcount: 17431)
DIS: Disease/Phenotype (termcount: 57676)
ACCN: ClinVar accession (termcount: 11118780)
VRID: External allele ID (termcount: 8319825)
TRID: Trait identifier (termcount: 43051)
PROP: Properties (termcount: 

In [371]:
clinvar_field_len = len(resp['einforesult']['dbinfo'][0]['fieldlist'])
clinvar_link_len = len(resp['einforesult']['dbinfo'][0]['linklist'])
print(f"{clinvar_field_len=}")
print(f"{clinvar_link_len=}")

clinvar_field_len=47
clinvar_link_len=12


### ESearch: PubMed를 비롯한 다양한 데이터베이스에서 검색 데이터 가져오기

Esearch는 Entrez 생태계에 있는 모든 DB에 대해 사용할 수 있습니다.  
**대표적 데이터베이스:**

* PubMed: 논문
* nucleotide: DNA / RNA 서열
* protein: 단백질 시퀀스 서열
* gene: 유전자 메타데이터
* ClinVar: 변이 정보

#### ESearch: PubMed

In [374]:
## ESearch: Find UIDs by keyword (usehistory=y)

query = '"LLM"[Title] AND bioinformatics'
params = {
    "db": "pubmed",
    "term": query,
    "retmax": 10,
}

pubmed_resp = eutils_request("esearch.fcgi", params)
print("Query:", query)
print("Count:", pubmed_resp["esearchresult"]["count"])
pubmed_uids = pubmed_resp["esearchresult"]["idlist"]
print("First UIDs:", pubmed_uids)

Query: "LLM"[Title] AND bioinformatics
Count: 20
First UIDs: ['41139312', '40973196', '40831495', '40601266', '40601260', '40586923', '40540387', '40209077', '40199828', '40194554']


In [375]:
pubmed_resp['esearchresult']['querytranslation']

'"LLM"[Title] AND ("bioinformatical"[All Fields] OR "bioinformatically"[All Fields] OR "computational biology"[MeSH Terms] OR ("computational"[All Fields] AND "biology"[All Fields]) OR "computational biology"[All Fields] OR "bioinformatic"[All Fields] OR "bioinformatics"[All Fields])'

#### ESearch: PMC

In [376]:
## ESearch: Find UIDs by keyword (usehistory=y)

query = '"LLM"[Title] AND bioinformatics'
params = {
    "db": "pmc",
    "term": query,
    "retmax": 10,
}

pmc_resp = eutils_request("esearch.fcgi", params)
print("Query:", query)
print("Count:", pmc_resp["esearchresult"]["count"])
pmc_uids = pmc_resp["esearchresult"]["idlist"]
print("First UIDs:", pmc_uids)

Query: "LLM"[Title] AND bioinformatics
Count: 30
First UIDs: ['12569577', '12539697', '12516316', '12488635', '12554096', '12485990', '12415252', '12351685', '12280092', '12263107']


In [377]:
pmc_resp['esearchresult']['querytranslation']

'"LLM"[Title] AND ("computational biology"[MeSH Terms] OR ("computational"[All Fields] AND "biology"[All Fields]) OR "computational biology"[All Fields] OR "bioinformatics"[All Fields])'

### EFetch: PubMed, PMC, Nucleotide 등 다양한 DB에서 특정 ID에 해당하는 상세 정보 반환

#### EFetch: Pubmed

full-text 부재, 초록 및 다양한 메타데이터 반환 가능

In [391]:
## EFetch: Full records (XML) for one PMID

import xmltodict

if pubmed_uids:
    first_uid = pubmed_uids[0]
    xml_text = eutils_request("efetch.fcgi",
                              {"db":"pubmed", "id": first_uid, "retmode":"xml"},
                              expect_json=False)
    doc = xmltodict.parse(xml_text)
    # Extract title (best-effort)
    try:
        art = doc["PubmedArticleSet"]["PubmedArticle"]["MedlineCitation"]["Article"]
        title = art.get("ArticleTitle", "")
        print("UID:", first_uid)
        print("ArticleTitle:", title)
    except Exception as e:
        print("Parsed keys:", list(doc.keys()))
else:
    print("No UIDs from ESearch step.")

UID: 41139312
ArticleTitle: BioLLMNet: enhancing RNA-interaction prediction with a specialized cross-LLM transformation network.


In [379]:
xml_text

'<?xml version="1.0" ?>\n<!DOCTYPE PubmedArticleSet PUBLIC "-//NLM//DTD PubMedArticle, 1st January 2025//EN" "https://dtd.nlm.nih.gov/ncbi/pubmed/out/pubmed_250101.dtd">\n<PubmedArticleSet>\n<PubmedArticle><MedlineCitation Status="MEDLINE" Owner="NLM" IndexingMethod="Automated"><PMID Version="1">41139312</PMID><DateCompleted><Year>2025</Year><Month>10</Month><Day>26</Day></DateCompleted><DateRevised><Year>2025</Year><Month>10</Month><Day>30</Day></DateRevised><Article PubModel="Print"><Journal><ISSN IssnType="Electronic">1477-4054</ISSN><JournalIssue CitedMedium="Internet"><Volume>26</Volume><Issue>5</Issue><PubDate><Year>2025</Year><Month>Aug</Month><Day>31</Day></PubDate></JournalIssue><Title>Briefings in bioinformatics</Title><ISOAbbreviation>Brief Bioinform</ISOAbbreviation></Journal><ArticleTitle>BioLLMNet: enhancing RNA-interaction prediction with a specialized cross-LLM transformation network.</ArticleTitle><ELocationID EIdType="pii" ValidYN="Y">bbaf549</ELocationID><ELocationID

In [380]:
document_keys = list(doc.keys())
document_articleset_keys = list(doc['PubmedArticleSet'].keys())
document_article_keys = list(doc["PubmedArticleSet"]["PubmedArticle"])
document_medlinecitation_keys = list(doc["PubmedArticleSet"]["PubmedArticle"]["MedlineCitation"].keys())
article_keys = list(doc["PubmedArticleSet"]["PubmedArticle"]["MedlineCitation"]["Article"].keys())
print(f"{document_keys=}")
print(f"{document_articleset_keys=}")
print(f"{document_article_keys=}")
print(f"{document_medlinecitation_keys=}")
print(f"{article_keys=}")

document_keys=['PubmedArticleSet']
document_articleset_keys=['PubmedArticle']
document_article_keys=['MedlineCitation', 'PubmedData']
document_medlinecitation_keys=['@Status', '@Owner', '@IndexingMethod', 'PMID', 'DateCompleted', 'DateRevised', 'Article', 'MedlineJournalInfo', 'ChemicalList', 'CitationSubset', 'MeshHeadingList', 'KeywordList']
article_keys=['@PubModel', 'Journal', 'ArticleTitle', 'ELocationID', 'Abstract', 'AuthorList', 'Language', 'PublicationTypeList']


In [295]:
art['Abstract']

{'AbstractText': 'Ribonucleic acids (RNAs) play a central role in cellular processes by interacting with proteins, small molecules, and other RNAs. Accurate prediction of these interactions is critical for understanding post-transcriptional regulation and advancing RNA-targeted therapeutics. However, existing computational methods are limited by their reliance on hand-crafted features, modality-specific architectures, and often require structural or physicochemical data, which are experimentally challenging to obtain and unavailable for many RNA molecules. These constraints hinder generalizability and fail to capture the complex, context-dependent semantics of RNA interactions. We present BioLLMNet, a unified sequence-only framework that leverages pretrained biological language models to encode rich, contextualized representations for both RNA molecules and their interacting partners, including proteins, small molecules, and other RNAs. Our key innovation is the introduction of a novel

#### EFetch: PMC

full-text 반환 가능, 등재 논문 수 적음

In [356]:
## EFetch: Full records (XML) for one PMID

if pmc_uids:
    first_uid = pmc_uids[0]
    xml_text = eutils_request("efetch.fcgi",
                              {"db":"pmc", "id": first_uid, "retmode":"xml"},
                              expect_json=False)
    doc = xmltodict.parse(xml_text)
    # Extract title (best-effort)
    
    art = doc["pmc-articleset"]["article"]["body"]["sec"][0]
    title = art['title']
    print("UID:", first_uid)
    print("Title:", title)

else:
    print("No UIDs from ESearch step.")

UID: 12569577
Title: Introduction


In [363]:
doc["pmc-articleset"]["article"]["body"]["sec"][3]

{'@id': 'j_jib-2024-0046_s_004',
 'label': '4',
 'title': 'Conclusions',
 'p': ['In our previous work, we developed Fcodes, a straightforward algorithm for kinship encoding. The Fcodes encoding method is highly flexible, preserving many properties of family structures, as demonstrated by its ability to estimate the inbreeding coefficient from two isolated Fcodes. To further our goal of making Fcodes a simple and accessible framework for managing kinship data, we introduced F-Tree, a graphical user interface. F-Tree represents a significant improvement in the accessibility of the algorithm, enabling its use in diverse research contexts and offering a powerful, flexible tool for managing genealogical data.',
  'Additionally, we explore the integration of LLMs with Fcodes to automatically infer family relationships from narrative texts. The promising results obtained suggest substantial potential for automating the processing of unstructured data, such as that found in medical records, ci

In [357]:
art

{'@id': 'j_jib-2024-0046_s_001',
 'label': '1',
 'title': 'Introduction',
 'p': ['As social creatures, we are part of a complex net of relationships that shape our personal growth, daily decisions, mental health, and overall quality of life. The closer the relationship, the greater its influence; that is why the study of family structures, aside from the biological standpoint, is so important for understanding social dynamics, history, and even epidemics, to cite a few examples.',
  {'italic': {'@toggle': 'yes',
    '#text': 'Thesaurus Principum Hac Aetate In Europa Viventium'},
   '#text': 'Registering and analyzing this type of information is complex. The first records of an encoding algorithm date from 1,590, where the Austrian historian Michaël Eytzinger designs an encoding system to document the royal European houses (). This system, later known as Ahnentafel, was widely used by genealogists for centuries. It assigns the number one to a reference person, doubling for the father an

In [346]:
document_keys = list(doc.keys())
document_articleset_keys = list(doc['pmc-articleset'].keys())
document_article_keys = list(doc["pmc-articleset"]["article"])
document_medlinecitation_keys = list(doc["pmc-articleset"]["article"]["body"].keys())
article_keys = list(doc["pmc-articleset"]["article"]["body"]["sec"])
print(f"{document_keys=}")
print(f"{document_articleset_keys=}")
print(f"{document_article_keys=}")
print(f"{document_medlinecitation_keys=}")
print(f"{article_keys=}")

document_keys=['pmc-articleset']
document_articleset_keys=['article']
document_article_keys=['@xmlns:mml', '@xmlns:ali', '@xml:lang', '@article-type', '@dtd-version', 'processing-meta', 'front', 'body', 'back']
document_medlinecitation_keys=['sec']
article_keys=[{'@id': 'j_jib-2024-0046_s_001', 'label': '1', 'title': 'Introduction', 'p': ['As social creatures, we are part of a complex net of relationships that shape our personal growth, daily decisions, mental health, and overall quality of life. The closer the relationship, the greater its influence; that is why the study of family structures, aside from the biological standpoint, is so important for understanding social dynamics, history, and even epidemics, to cite a few examples.', {'italic': {'@toggle': 'yes', '#text': 'Thesaurus Principum Hac Aetate In Europa Viventium'}, '#text': 'Registering and analyzing this type of information is complex. The first records of an encoding algorithm date from 1,590, where the Austrian historia

#### 이제 데이터 전처리만 배우면 데이터를 API를 통해 가져와서 전처리하여 내가 사용할 수 있는 데이터셋을 구축하는 것이 가능해집니다

### References

- NCBI Bookshelf — *A General Introduction to the E-utilities* (NBK25497)
- E-utilities Base: https://eutils.ncbi.nlm.nih.gov/entrez/eutils/
- PubMed Help: https://pubmed.ncbi.nlm.nih.gov/help/
- E-utilities API parameters: https://www.ncbi.nlm.nih.gov/books/NBK25499/

### 관련 있는 Github 오픈소스 패키지

- [scholarly-python-package/scholarly](https://github.com/scholarly-python-package/scholarly)
- [jannisborn/paperscraper](https://github.com/jannisborn/paperscraper)
- [neuroquery/pubget](https://github.com/neuroquery/pubget)